In [11]:
# Imports we need.
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

import tensor2tensor as t2t

In [5]:
# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

In [6]:
# Other setup
Modes = tf.estimator.ModeKeys

In [7]:
# Setup some directories
data_dir = os.path.expanduser("~/t2t/data")
tmp_dir = os.path.expanduser("~/t2t/tmp")
train_dir = os.path.expanduser("~/t2t/train")
checkpoint_dir = os.path.expanduser("~/t2t/checkpoints")
tf.gfile.MakeDirs(data_dir)
tf.gfile.MakeDirs(tmp_dir)
tf.gfile.MakeDirs(train_dir)
tf.gfile.MakeDirs(checkpoint_dir)
gs_data_dir = "gs://tensor2tensor-data"
gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"

In [8]:
# A Problem is a dataset together with some fixed pre-processing.
# It could be a translation dataset with a specific tokenization,
# or an image dataset with a specific resolution.
#
# There are many problems available in Tensor2Tensor
print("\n".join(problems.available()))

algorithmic_addition_binary40
algorithmic_addition_decimal40
algorithmic_cipher_shift200
algorithmic_cipher_shift5
algorithmic_cipher_vigenere200
algorithmic_cipher_vigenere5
algorithmic_identity_binary40
algorithmic_identity_decimal40
algorithmic_identity_vocab95_train20_eval30
algorithmic_math_deepmind_all
algorithmic_math_two_variables
algorithmic_multiplication_binary40
algorithmic_multiplication_decimal40
algorithmic_reverse_binary40
algorithmic_reverse_binary40_test
algorithmic_reverse_decimal40
algorithmic_reverse_nlplike32k
algorithmic_reverse_nlplike8k
algorithmic_shift_decimal40
algorithmic_sort_problem
audio_timit_characters_tune
audio_timit_tokens8k_test
audio_timit_tokens8k_tune
babi_qa_concat_all_tasks_10k
babi_qa_concat_all_tasks_1k
babi_qa_concat_task10_10k
babi_qa_concat_task10_1k
babi_qa_concat_task11_10k
babi_qa_concat_task11_1k
babi_qa_concat_task12_10k
babi_qa_concat_task12_1k
babi_qa_concat_task13_10k
babi_qa_concat_task13_1k
babi_qa_concat_task14_10k
babi_qa_conc

In [9]:
# Fetch the MNIST problem
mnist_problem = problems.problem("image_mnist")
# The generate_data method of a problem will download data and process it into
# a standard format ready for training and evaluation.
mnist_problem.generate_data(data_dir, tmp_dir)

INFO:tensorflow:Not downloading, file already found: /home/statisticallyfit/t2t/tmp/train-images-idx3-ubyte.gz


INFO:tensorflow:Not downloading, file already found: /home/statisticallyfit/t2t/tmp/train-images-idx3-ubyte.gz


INFO:tensorflow:Not downloading, file already found: /home/statisticallyfit/t2t/tmp/train-labels-idx1-ubyte.gz


INFO:tensorflow:Not downloading, file already found: /home/statisticallyfit/t2t/tmp/train-labels-idx1-ubyte.gz


INFO:tensorflow:Not downloading, file already found: /home/statisticallyfit/t2t/tmp/t10k-images-idx3-ubyte.gz


...

INFO:tensorflow:Skipping generator because outputs files exists at ['/home/statisticallyfit/t2t/data/image_mnist-unshuffled-train-00000-of-00010', '/home/statisticallyfit/t2t/data/image_mnist-unshuffled-train-00001-of-00010', '/home/statisticallyfit/t2t/data/image_mnist-unshuffled-train-00002-of-00010', '/home/statisticallyfit/t2t/data/image_mnist-unshuffled-train-00003-of-00010', '/home/statisticallyfit/t2t/data/image_mnist-unshuffled-train-00004-of-00010', '/home/statisticallyfit/t2t/data/image_mnist-unshuffled-train-00005-of-00010', '/home/statisticallyfit/t2t/data/image_mnist-unshuffled-train-00006-of-00010', '/home/statisticallyfit/t2t/data/image_mnist-unshuffled-train-00007-of-00010', '/home/statisticallyfit/t2t/data/image_mnist-unshuffled-train-00008-of-00010', '/home/statisticallyfit/t2t/data/image_mnist-unshuffled-train-00009-of-00010']


INFO:tensorflow:Skipping generator because outputs files exists at ['/home/statisticallyfit/t2t/data/image_mnist-unshuffled-dev-00000-of-00001']


INFO:tensorflow:Skipping generator because outputs files exists at ['/home/statisticallyfit/t2t/data/image_mnist-unshuffled-dev-00000-of-00001']


INFO:tensorflow:Skipping shuffle because output files exist


INFO:tensorflow:Skipping shuffle because output files exist


In [13]:
# Now let's see the training MNIST data as Tensors.
mnist_example = tfe.Iterator(mnist_problem.dataset(Modes.TRAIN, data_dir)).next()
image = mnist_example["inputs"]
label = mnist_example["targets"]

INFO:tensorflow:Reading data files from /home/statisticallyfit/t2t/data/image_mnist-train*


INFO:tensorflow:Reading data files from /home/statisticallyfit/t2t/data/image_mnist-train*


INFO:tensorflow:partition: 0 num_data_files: 10


INFO:tensorflow:partition: 0 num_data_files: 10


Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [14]:
plt.imshow(image.numpy()[:, :, 0].astype(np.float32), cmap=plt.get_cmap('gray'))
print("Label: %d" % label.numpy())

Label: 4


In [16]:
# TRANSLATE FROM ENGLISH TO GERMAN WITH PRE_TRAINED MODEL

# Fetch the problem
ende_problem = problems.problem("translate_ende_wmt32k")

# Copy the vocab file locally so we can encode inputs and decode model outputs
# All vocabs are stored on GCS
vocab_name = "vocab.translate_ende_wmt32k.32768.subwords"
vocab_file = os.path.join(gs_data_dir, vocab_name)
!gsutil cp {vocab_file} {data_dir}

# Get the encoders from the problem
encoders = ende_problem.feature_encoders(data_dir)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None): 
    """Input str to features dict, ready for inference""" 
    inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
    return {"inputs": batch_inputs}

def decode(integers): 
    """List of ints to str""" 
    integers = list(np.squeeze(integers)) 
    if 1 in integers: 
        integers = integers[:integers.index(1)] 
    return encoders["inputs"].decode(np.squeeze(integers))

/bin/sh: 1: gsutil: not found


ValueError: File /home/statisticallyfit/t2t/data/vocab.translate_ende_wmt32k.32768.subwords not found